In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import OneClassSVM
from sklearn.mixture import GaussianMixture
from sklearn.metrics import roc_auc_score, average_precision_score

## 1- Load Part D Dataset

In [ ]:
# 1. Read Medicare Part D data and LEIE data.
combined_df = pd.read_csv("outputs/Combined_LEIE_Medicare_2017_2019_DOWNSIZED_1mil.csv")


In [4]:
combined_df.head()

,Tot_Benes_sum,Tot_Benes_mean,Tot_Benes_median,Tot_Benes_std,Tot_Benes_min,Tot_Benes_max,Tot_Clms_sum,Tot_Clms_mean,Tot_Clms_median,Tot_Clms_std,...,"Prscrbr_Type_Technician, Pathology",Prscrbr_Type_Technician/Technologist,Prscrbr_Type_Thoracic Surgery,Prscrbr_Type_Thoracic Surgery (Cardiothoracic Vascular Surgery),Prscrbr_Type_Undefined Physician type,Prscrbr_Type_Undersea and Hyperbaric Medicine,Prscrbr_Type_Unknown Supplier/Provider Specialty,Prscrbr_Type_Urology,Prscrbr_Type_Vascular Surgery,Prscrbr_Type_Voluntary Health or Charitable Agency
0,291.0,29.10,23.5,17.571758,11.0,63.0,405.0,33.750000,23.5,20.150006,...,False,False,False,False,False,False,False,False,False,False
1,36.0,12.00,11.0,1.732051,11.0,14.0,124.0,17.714286,14.0,7.973169,...,False,False,False,False,False,False,False,False,False,False
2,24.0,12.00,12.0,1.414214,11.0,13.0,272.0,20.923077,16.0,14.952489,...,False,False,False,False,False,False,False,False,False,False
3,71.0,17.75,18.5,4.193249,12.0,22.0,238.0,21.636364,18.0,10.726348,...,False,False,False,False,False,False,False,False,False,False
4,83.0,16.60,12.0,7.436397,11.0,27.0,679.0,24.250000,20.0,14.410965,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
# 2. Inspect column names to avoid KeyError. 
print("Combined columns:", combined_df.columns.tolist())

In [ ]:
# 3. Quick sanity check: 
print("\n=== Medicare Merged Dataset ===")
print("Merged Dataset shape:", combined_df.shape)
print("FraudFlag distribution:\n", combined_df["TARGET"].value_counts())

## 2- Normalization and Mapping of Features

In [ ]:
# 1. We want to select only truly numeric features (excluding identifiers).
#    Using select_dtypes prevents KeyError that arises if you manually list wrong names.
numeric_cols = combined_df.select_dtypes(include=[np.number]).columns.tolist()
print("\nAll numeric columns (including IDs & target):\n", numeric_cols)


All numeric columns (including IDs & target):
 ['Tot_Benes_sum', 'Tot_Benes_mean', 'Tot_Benes_median', 'Tot_Benes_std', 'Tot_Benes_min', 'Tot_Benes_max', 'Tot_Clms_sum', 'Tot_Clms_mean', 'Tot_Clms_median', 'Tot_Clms_std', 'Tot_Clms_min', 'Tot_Clms_max', 'Tot_30day_Fills_sum', 'Tot_30day_Fills_mean', 'Tot_30day_Fills_median', 'Tot_30day_Fills_std', 'Tot_30day_Fills_min', 'Tot_30day_Fills_max', 'Tot_Day_Suply_sum', 'Tot_Day_Suply_mean', 'Tot_Day_Suply_median', 'Tot_Day_Suply_std', 'Tot_Day_Suply_min', 'Tot_Day_Suply_max', 'Tot_Drug_Cst_sum', 'Tot_Drug_Cst_mean', 'Tot_Drug_Cst_median', 'Tot_Drug_Cst_std', 'Tot_Drug_Cst_min', 'Tot_Drug_Cst_max']


### Normalization

In [ ]:
# 2. From these numeric columns, drop 'NPI' (identifier) and 'FraudFlag' (target).
features_list = [col for col in numeric_cols if col not in ["npi", "TARGET"]]
print("\nFeatures selected for modeling (numeric_cols without 'NPI' and 'FraudFlag'):\n", features_list)

In [ ]:
# 1. Create a StandardScaler instance to normalize these numeric features.
scaler_med = StandardScaler()


# 2. Fit & transform these features in merged_df, overwriting them with scaled values.
combined_df[features_list] = scaler_med.fit_transform(combined_df[features_list])

In [ ]:
# 3- Define mappings for common boolean-like values
bool_mappings = {
    'True': 1, 'False': 0,
    'true': 1, 'false': 0,
    True: 1, False: 0,
    'Positive': 1, 'Negative': 0,
    'Yes': 1, 'No': 0
}

# 4- Identify boolean or boolean-like columns
bool_cols = []

for col in combined_df.columns:
    col_data = combined_df[col]
    if col_data.dtype == 'bool':
        bool_cols.append(col)
    elif col_data.dtype == 'object' and set(col_data.dropna().unique()).issubset(set(bool_mappings.keys())):
        bool_cols.append(col)
    elif pd.api.types.is_numeric_dtype(col_data) and set(col_data.dropna().unique()).issubset({0, 1}):
        bool_cols.append(col)

# 5- Convert values in those columns using the mapping
for col in bool_cols:
    combined_df[col] = combined_df[col].map(bool_mappings).astype('int')

Convert categorical bool to int


In [15]:
combined_df.head()

,Tot_Benes_sum,Tot_Benes_mean,Tot_Benes_median,Tot_Benes_std,Tot_Benes_min,Tot_Benes_max,Tot_Clms_sum,Tot_Clms_mean,Tot_Clms_median,Tot_Clms_std,...,"Prscrbr_Type_Technician, Pathology",Prscrbr_Type_Technician/Technologist,Prscrbr_Type_Thoracic Surgery,Prscrbr_Type_Thoracic Surgery (Cardiothoracic Vascular Surgery),Prscrbr_Type_Undefined Physician type,Prscrbr_Type_Undersea and Hyperbaric Medicine,Prscrbr_Type_Unknown Supplier/Provider Specialty,Prscrbr_Type_Urology,Prscrbr_Type_Vascular Surgery,Prscrbr_Type_Voluntary Health or Charitable Agency
0,291.0,29.10,23.5,17.571758,11.0,63.0,405.0,33.750000,23.5,20.150006,...,0,0,0,0,0,0,0,0,0,0
1,36.0,12.00,11.0,1.732051,11.0,14.0,124.0,17.714286,14.0,7.973169,...,0,0,0,0,0,0,0,0,0,0
2,24.0,12.00,12.0,1.414214,11.0,13.0,272.0,20.923077,16.0,14.952489,...,0,0,0,0,0,0,0,0,0,0
3,71.0,17.75,18.5,4.193249,12.0,22.0,238.0,21.636364,18.0,10.726348,...,0,0,0,0,0,0,0,0,0,0
4,83.0,16.60,12.0,7.436397,11.0,27.0,679.0,24.250000,20.0,14.410965,...,0,0,0,0,0,0,0,0,0,0


## 3- Data Splitting

In [ ]:
# 1. Separate into X_med (feature matrix) and y_med (target vector)
X_med = combined_df.drop(columns=["TARGET"])
y_med = combined_df["TARGET"]

In [19]:
y_med

0         NOT_FRAUD
1         NOT_FRAUD
2         NOT_FRAUD
3         NOT_FRAUD
4         NOT_FRAUD
            ...    
999995    NOT_FRAUD
999996    NOT_FRAUD
999997    NOT_FRAUD
999998    NOT_FRAUD
999999    NOT_FRAUD
Name: TARGET, Length: 1000000, dtype: object

In [ ]:
# 2- Map target variables from string to int
y_med = y_med.map({"NOT_FRAUD": 0, "FRAUD": 1})
fraud_values = y_med[y_med == 1]
fraud_values

In [23]:
y_med

0         0
1         0
2         0
3         0
4         0
         ..
999995    0
999996    0
999997    0
999998    0
999999    0
Name: TARGET, Length: 1000000, dtype: int64

## 4- Cross Validation

In [ ]:
# 1. Use StratifiedKFold to keep class imbalance roughly equal across folds.
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
repeats = 10

In [ ]:
results_med = {
    'majority': {
        'OCSVM': [],
        'OCSVM_Sigmoid': [],
        'OCSVM_Isotonic': [],
        'GMM': [],
        'GMM_Sigmoid': []
    },
    'minority': {
        'OCSVM': [],
        'OCSVM_Sigmoid': [],
        'OCSVM_Isotonic': [],
        'GMM': [],
        'GMM_Sigmoid': []
    },
}

## 5- Calibration

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.isotonic import IsotonicRegression

def calibrate_scores(scores, y_true, method="sigmoid"):
    if method == "sigmoid":
        calibrator = LogisticRegression(solver="lbfgs")
    elif method == "isotonic":
        calibrator = IsotonicRegression(out_of_bounds="clip")
    else:
        raise ValueError("Unknown calibration method.")

    calibrator.fit(scores.reshape(-1, 1), y_true)
    return calibrator


## 6- Model Training

In [ ]:
# for memory efficiency
del combined_df

In [ ]:
# MEDICARE: MAJORITY (Only Normal Providers) VS MINORITY (Only Excluded/Fraud) TRAINING

for r in range(repeats):
    print(f"\n--- Medicare: Repeat {r+1}/{repeats} ---")
    for fold, (train_idx, test_idx) in enumerate(skf.split(X_med, y_med), start=1):
        print(f"fold: {fold}")
        # Split X_med and y_med into training/testing for this fold.
        X_train_m, X_test_m = X_med.values[train_idx], X_med.values[test_idx]
        y_train_m, y_test_m = y_med.values[train_idx], y_med.values[test_idx]

        # ---------------------------------
        # 1) Majority training: (y_train_m == 0)
        # ---------------------------------
        X_train_major_m = X_train_m[y_train_m == 0]  # Normal providers



        # 1st Model
        # -- One-Class SVM (Majority) --
        ocsvm_major_m = OneClassSVM(kernel="rbf", nu=0.01, gamma="scale")
        print("training is started!")
        ocsvm_major_m.fit(X_train_major_m)


        # Get anomaly scores
        svm_scores_raw = ocsvm_major_m.decision_function(X_test_m)
        svm_anomaly = -svm_scores_raw

        # Calibrate
        calibrator_sigmoid = calibrate_scores(svm_anomaly, y_test_m, method="sigmoid")
        calibrator_isotonic = calibrate_scores(svm_anomaly, y_test_m, method="isotonic")

        # Get calibrated probabilities
        prob_sigmoid = calibrator_sigmoid.predict_proba(svm_anomaly.reshape(-1, 1))[:, 1] if hasattr(calibrator_sigmoid, "predict_proba") else calibrator_sigmoid.predict(svm_anomaly.reshape(-1, 1))
        prob_isotonic = calibrator_isotonic.predict(svm_anomaly.reshape(-1, 1))

        # Evaluate
        auc_sigmoid = roc_auc_score(y_test_m, prob_sigmoid)
        aupr_sigmoid = average_precision_score(y_test_m, prob_sigmoid)
        auc_isotonic = roc_auc_score(y_test_m, prob_isotonic)
        aupr_isotonic = average_precision_score(y_test_m, prob_isotonic)

        results_med['majority']['OCSVM_Sigmoid'].append((auc_sigmoid, aupr_sigmoid))
        results_med['majority']['OCSVM_Isotonic'].append((auc_isotonic, aupr_isotonic))

        print(f"Majority OCSVM + Sigmoid: AUC={auc_sigmoid:.4f}, AUPR={aupr_sigmoid:.4f}")
        print(f"Majority OCSVM + Isotonic: AUC={auc_isotonic:.4f}, AUPR={aupr_isotonic:.4f}")



        # 2nd Model
        # -- One-Class GMM (Majority) --
        gmm_major_m = GaussianMixture(n_components=1, random_state=42)
        gmm_major_m.fit(X_train_major_m)

        # Get raw scores and convert to anomaly scores
        gmm_scores_raw_major_gmm = gmm_major_m.score_samples(X_test_m)
        gmm_anomaly_major_gmm = -gmm_scores_raw_major_gmm

        # Calibrate using sigmoid and isotonic
        calibrator_sigmoid_major_gmm = calibrate_scores(gmm_anomaly_major_gmm, y_test_m, method="sigmoid")

        # Get calibrated probabilities
        # prob_sigmoid_major_gmm = (
        #     calibrator_sigmoid_major_gmm.predict_proba(gmm_anomaly_major_gmm.reshape(-1, 1))[:, 1]
        #     if hasattr(calibrator_sigmoid_major_gmm, "predict_proba")
        #     else calibrator_sigmoid_major_gmm.predict(gmm_anomaly_major_gmm.reshape(-1, 1))
        # )

        prob_sigmoid_major_gmm = calibrator_sigmoid_major_gmm.predict_proba(gmm_anomaly_major_gmm.reshape(-1, 1))[:, 1]


        # Evaluate calibrated results
        auc_sigmoid_major_gmm = roc_auc_score(y_test_m, prob_sigmoid_major_gmm)
        aupr_sigmoid_major_gmm = average_precision_score(y_test_m, prob_sigmoid_major_gmm)


        # Save results
        results_med['majority']['GMM_Sigmoid'].append((auc_sigmoid_major_gmm, aupr_sigmoid_major_gmm))

        # Print results
        print(f"Majority GMM + Sigmoid: AUC={auc_sigmoid_major_gmm:.4f}, AUPR={aupr_sigmoid_major_gmm:.4f}")











        # ---------------------------------
        # 2) Minority training: (y_train_m == 1)
        # ---------------------------------
        X_train_minor_m = X_train_m[y_train_m == 1]

        if len(X_train_minor_m) > 0:


            # -- One-Class SVM (Minority) --
            ocsvm_minor_m = OneClassSVM(kernel="rbf", nu=0.1, gamma="scale")
            print("training is started!")
            ocsvm_minor_m.fit(X_train_minor_m)

            # Get anomaly scores
            svm_scores_raw_minor_svm = ocsvm_minor_m.decision_function(X_test_m)
            svm_anomaly_minor_svm = -svm_scores_raw_minor_svm

            # Calibrate
            calibrator_sigmoid_minor_svm = calibrate_scores(svm_anomaly_minor_svm, y_test_m, method="sigmoid")
            calibrator_isotonic_minor_svm = calibrate_scores(svm_anomaly_minor_svm, y_test_m, method="isotonic")

            # Get calibrated probabilities
            prob_sigmoid_minor_svm = (
                calibrator_sigmoid_minor_svm.predict_proba(svm_anomaly_minor_svm.reshape(-1, 1))[:, 1]
                if hasattr(calibrator_sigmoid_minor_svm, "predict_proba")
                else calibrator_sigmoid_minor_svm.predict(svm_anomaly_minor_svm.reshape(-1, 1))
            )

            prob_isotonic_minor_svm = calibrator_isotonic_minor_svm.predict(svm_anomaly_minor_svm.reshape(-1, 1))

            # Evaluate
            auc_sigmoid_minor_svm = roc_auc_score(y_test_m, prob_sigmoid_minor_svm)
            aupr_sigmoid_minor_svm = average_precision_score(y_test_m, prob_sigmoid_minor_svm)
            auc_isotonic_minor_svm = roc_auc_score(y_test_m, prob_isotonic_minor_svm)
            aupr_isotonic_minor_svm = average_precision_score(y_test_m, prob_isotonic_minor_svm)

            # Store results
            results_med['minority']['OCSVM_Sigmoid'].append((auc_sigmoid_minor_svm, aupr_sigmoid_minor_svm))
            results_med['minority']['OCSVM_Isotonic'].append((auc_isotonic_minor_svm, aupr_isotonic_minor_svm))

            # Print results
            print(f"Minority OCSVM (Minority) + Sigmoid: AUC={auc_sigmoid_minor_svm:.4f}, AUPR={aupr_sigmoid_minor_svm:.4f}")
            print(f"Minority OCSVM (Minority) + Isotonic: AUC={auc_isotonic_minor_svm:.4f}, AUPR={aupr_isotonic_minor_svm:.4f}")





            # -- One-Class GMM (Minority) --
            gmm_minor_m = GaussianMixture(n_components=1, random_state=42)
            gmm_minor_m.fit(X_train_minor_m)


            # Get raw scores and convert to anomaly scores (negate log-likelihood)
            gmm_scores_raw_minor_gmm = gmm_minor_m.score_samples(X_test_m)
            gmm_anomaly_minor_gmm = -gmm_scores_raw_minor_gmm

            # Calibrate using sigmoid (you can add isotonic similarly if needed)
            calibrator_sigmoid_minor_gmm = calibrate_scores(gmm_anomaly_minor_gmm, y_test_m, method="sigmoid")

            # Get calibrated probabilities
            prob_sigmoid_minor_gmm = (
                calibrator_sigmoid_minor_gmm.predict_proba(gmm_anomaly_minor_gmm.reshape(-1, 1))[:, 1]
                if hasattr(calibrator_sigmoid_minor_gmm, "predict_proba")
                else calibrator_sigmoid_minor_gmm.predict(gmm_anomaly_minor_gmm.reshape(-1, 1))
            )

            # Get calibrated probabilities
            # prob_sigmoid_minor_gmm = calibrator_sigmoid_minor_gmm.predict_proba(gmm_anomaly_minor_gmm.reshape(-1, 1))[:, 1]


            # Evaluate calibrated results
            auc_sigmoid_minor_gmm = roc_auc_score(y_test_m, prob_sigmoid_minor_gmm)
            aupr_sigmoid_minor_gmm = average_precision_score(y_test_m, prob_sigmoid_minor_gmm)

            # Save results
            results_med['minority']['GMM_Sigmoid'].append((auc_sigmoid_minor_gmm, aupr_sigmoid_minor_gmm))

            # Print results
            print(f"Minority GMM + Sigmoid: AUC={auc_sigmoid_minor_gmm:.4f}, AUPR={aupr_sigmoid_minor_gmm:.4f}")
        else:
            # If no excluded providers in this fold’s training split, skip minority training.
            continue

    print(f"--- Finished Medicare Repeat {r+1} ---")

## 7- Evaluation

In [ ]:
def compute_mean_scores(score_list):
    """
    Given a list of (AUC, AUPRC) tuples, return the average AUC and AUPRC.
    """
    arr = np.array(score_list)   # shape = (num_experiments, 2)
    return arr.mean(axis=0)      # returns (mean_auc, mean_auprc)


In [ ]:
for group in results_med:
    print(f'\n>>> {group.upper()}')
    for method in results_med[group]:
        scores = results_med[group][method]
        if scores:
            mean_auc, mean_aupr = compute_mean_scores(scores)
            print(f'{method:20s} | AUC: {mean_auc:.4f} | AUPRC: {mean_aupr:.4f}')
        else:
            print(f'{method:20s} | No scores available.')